In [24]:
import warnings
warnings.filterwarnings('ignore')

import re
import time
import sqlite3
import requests
from pandas import DataFrame
from bs4 import BeautifulSoup
from multiprocessing import Pool
import datetime

In [2]:
def GetPageContent(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    res = requests.get(url, headers=headers)
    content = BeautifulSoup(res.text)
    return content

In [3]:
def Parse(content):
    #soup = GetPageContent('https://www.mobile01.com/' + url)
    #origin = soup.find('div', {'class':'single-post-content'}) # 文章內文在 <div class="single-post-content"> 底下
    origin = content
    if origin:
        content = str(origin)
        # replace <br>, <br\> and '\n' with a whitespace########
        content = re.sub("<br\s*>", " ", content, flags=re.I)  #
        content = re.sub("<br\s*/>", " ", content, flags=re.I) #
        content = re.sub("\n+", " ", content, flags=re.I)      #
        ########################################################
        # remove hyperlink
        content = re.sub("<a\s+[^<>]+>(?P<aContent>[^<>]+?)</a>", "\g<aContent>", content, flags=re.I)
        content = BeautifulSoup(content)
        content = ' '.join(content.text.lstrip().rstrip().split())
    else:
        content = 'None'

    return content

In [78]:
def GetPageReviews(url):
    All_page_Contents = GetPageContent(url)
    Review_list = All_page_Contents.find_all('div',{'class':'l-articlePage'})
    
    resp = list()
    
    for i in range(len(Review_list)):
        date = datetime.datetime.strptime((Review_list[0].find('span',{'class':'o-fNotes o-fSubMini'})).text, '%Y-%m-%d %H:%M')
        topic = All_page_Contents.find('h2',{'class':'t2'}).text
        review = Parse(Review_list[i].find('article').text) 
        id = Parse(Review_list[i])
        
        resp.append({
            'date':date,
            'topic':topic,
            'review':review,
            'id':id,
            'url':url
        })
        
    return resp

In [83]:
def Save2Excel(posts):
    topics = [entry['topic'] for entry in posts]
    links = [entry['url'] for entry in posts]
    dates = [entry['date'] for entry in posts]
    authors = [entry['id'] for entry in posts]
    contents = [entry['review'] for entry in posts]
    df = DataFrame({
        '主題':topics,
        'URL':links,
        'date': dates,
        'id':authors,
        '留言': contents
        })
    df.to_excel('data.xlsx', sheet_name='sheet1', index=False, columns=['發文周','date','Series','主題','id','留言','留言好感度','留言Feature','URL','留言型號','非競品品牌','非競品型號','文章好感度','文章feature'])

In [84]:
url = 'https://www.mobile01.com/topicdetail.php?f=233&t=5863040'

In [85]:
K =  GetPageReviews(url)

In [86]:
Save2Excel(K)

In [58]:
All_page_Contents = GetPageContent(url)

In [59]:
Review_list = All_page_Contents.find_all('div',{'class':'l-articlePage'})

In [67]:
(Review_list[0].find('span',{'class':'o-fNotes o-fSubMini'})).text

'2019-08-04 14:44'

In [26]:
date_time_obj = datetime.datetime.strptime(J[0].text, '%Y-%m-%d %H:%M')

In [27]:
date_time_obj

datetime.datetime(2019, 8, 4, 14, 44)